![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=SocialStudies/HansardAnalysis/hansard-analysis.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Sunset and Sunrise Analysis

Welcome to the wonderful world of Sunrises and Sunsets! In this vibrant notebook, we embark on breathtaking journey through the captivating beauty of nature's daily masterpieces: sunrises and sunsets.

Sunrise and sunset data was collected through the [Sunset and Sunrise](https://sunrise-sunset.org/api) api, which provides sunset and sunrise times when given a specific latitude and longitude. It is a simple REST api which requires a GET request from [here](https://api.sunrise-sunset.org/json). Other sources for sunrise and sunset times, such as the sunset and sunrise times of Calgary, can be found [here](https://www.timeanddate.com/sun/canada/calgary)

In [84]:
import requests
from datetime import *
from dateutil import tz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [85]:
lat= 51.0447
lng= -114.0719

query = {'lat': lat, 'lng': lng, 'date': 'today'}
response = requests.get(url = 'https://api.sunrise-sunset.org/json?', params=query)

data = response.json()
print(data)
df = pd.json_normalize(data['results'])
display(df)

{'results': {'sunrise': '11:39:43 AM', 'sunset': '3:25:45 AM', 'solar_noon': '7:32:44 PM', 'day_length': '15:46:02', 'civil_twilight_begin': '11:00:10 AM', 'civil_twilight_end': '4:05:17 AM', 'nautical_twilight_begin': '10:03:11 AM', 'nautical_twilight_end': '5:02:16 AM', 'astronomical_twilight_begin': '8:35:22 AM', 'astronomical_twilight_end': '6:30:05 AM'}, 'status': 'OK'}


,sunrise,sunset,solar_noon,day_length,civil_twilight_begin,civil_twilight_end,nautical_twilight_begin,nautical_twilight_end,astronomical_twilight_begin,astronomical_twilight_end
0,11:39:43 AM,3:25:45 AM,7:32:44 PM,15:46:02,11:00:10 AM,4:05:17 AM,10:03:11 AM,5:02:16 AM,8:35:22 AM,6:30:05 AM


Looking at the data in the _json_ format and the _dataframe_ format, we have multiple things to take note of. In particular, we see that we have access to the time of `sunrise` and `sunset` given a particular latitude and longitude. We also have access to `day_length` which highlights the length of a particular date. However, one particular issue with this dataset is that times are represented in the **UTC timezone**. Let's keep mind of this and _transform_ our data later.

In [86]:
sunrise = data['results']['sunrise']
sunset = data['results']['sunset']
print(f"Given latitude {lat} and longitude {lng}, the corresponding sunrise and sunset times are {sunrise} and {sunset} in the UTC timezone.")

Given latitude 51.0447 and longitude -114.0719, the corresponding sunrise and sunset times are 11:39:43 AM and 3:25:45 AM in the UTC timezone.


`Longitude` controls the directions of _east_ and _west_, which generally alters timezones, leading to leaps of times. `Latitude` controls the directions of _north_ and _south_, which affects the length of day and night. The rule is that the further away someone is from the equator, the greater the differences are between nighttime and daytime.

Remember our issue earlier regarding timezones? Let's solve that by transforming our time from the __UTC timezone__ to the __MST timezone__. This is a fairly simple change, as the __UTC timezone__ is 6 hours ahead of the __MST timezome__. By going back 6 hours, we are able to obtain times in our timezone. 

Later on, when you input your own `longitude` and `latitude` values, keep in mind your particular timezone in order to differentiate from the __UTC timezone__. We'll also find a solution to finding your particular timezone based off inputted `longitude` and `latitude` values. 

In [87]:
def time_convert(time, timezone:int):
    # Parse the time string into a datetime object
    t = datetime.strptime(time, '%I:%M:%S %p') - timedelta(hours=timezone)
    # Finding your timezone through latitude and longitude values

    # Format the datetime object into a 24-hour time string
    format = '%H:%M:%S'
    return t.strftime(format)


print(time_convert(sunrise, 6))

21:25:45


In [88]:
# params format: latitude, longitude, date (YYYY-MM-DD)
params = {"lat":40.779659, "lng":-73.968995, "date":"2023-01-17"}

f = "https://api.sunrise-sunset.org/json"

def sunrisesunset(f):
    data = requests.get(f, params=params)
    data = json.loads(data.text)
    data = data["results"]
    return (data["sunrise"], data["sunset"], data["day_length"])

my_data = (sunrisesunset(f))
sunrise_times = my_data[0]
sunrise_times

'12:15:26 PM'

In [89]:
total_info = {"date": []}

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2023, 4, 17)
end_date = date(2023, 5, 17)

for single_date in daterange(start_date, end_date):
    total_info["date"].append(single_date.strftime("%Y-%m-%d"))
data = pd.DataFrame(total_info)

display(data)

,date
0,2023-04-17
1,2023-04-18
2,2023-04-19
3,2023-04-20
4,2023-04-21
5,2023-04-22
6,2023-04-23
7,2023-04-24
8,2023-04-25
9,2023-04-26


In [90]:
# Alter these longitude and latitude values below

longitude = 13.358
latitude = 52.5061


In [91]:
# params format: latitude, longitude, date (YYYY-MM-DD)
sunset_info = {"sunset": []}
sunrise_info = {"sunrise": []}
day_length = {"day_length": []}

def sunrisesunset(lat, lng, date):
    params = {"lat":lat, "lng":lng, "date":date}
    data = requests.get("https://api.sunrise-sunset.org/json", params=params)
    data = json.loads(data.text)
    data = data["results"]
    return data["sunrise"], data["sunset"], data["day_length"]


for i in range(len(data)):
    temp, temp1, temp2 = sunrisesunset(longitude, latitude, total_info['date'][i])
    sunset_info['sunset'].append(time_convert(temp, 6))
    sunrise_info['sunrise'].append(time_convert(temp1, 6))
    day_length['day_length'].append(temp2)

sunset_info = pd.DataFrame(sunset_info)
sunrise_info = pd.DataFrame(sunrise_info)
day_length = pd.DataFrame(day_length)

In [92]:
data = data.join(sunset_info['sunset'])
data = data.join(sunrise_info["sunrise"])
data = data.join(day_length['day_length'])
display(data)

,date,sunset,sunrise,day_length
0,2023-04-17,20:15:00,08:44:15,12:29:15
1,2023-04-18,20:14:26,08:44:23,12:29:57
2,2023-04-19,20:13:51,08:44:30,12:30:39
3,2023-04-20,20:13:18,08:44:38,12:31:20
4,2023-04-21,20:12:45,08:44:47,12:32:02
5,2023-04-22,20:12:12,08:44:55,12:32:43
6,2023-04-23,20:11:40,08:45:04,12:33:24
7,2023-04-24,20:11:09,08:45:13,12:34:04
8,2023-04-25,20:10:38,08:45:23,12:34:45
9,2023-04-26,20:10:08,08:45:32,12:35:24
